<a href="https://colab.research.google.com/github/IosuAV/ibm_practice/blob/main/Space_X_Falcon_9_First_Stage_Landing_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia

Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`:
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [ ]:
# !pip3 install beautifulsoup4
# !pip3 install requests

In [ ]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [ ]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()

    colunm_name = ' '.join(row.contents)

    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`

In [ ]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [ ]:
# use requests.get() method with the provided static_url
# assign the response to a object
data_falcon9 = requests.get(static_url)

Create a `BeautifulSoup` object from the HTML `response`

In [ ]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
soup = BeautifulSoup(data_falcon9.text)

Print the page title to verify if the `BeautifulSoup` object was created properly

In [ ]:
# Use soup.title attribute
soup.title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

### TASK 2: Extract all column/variable names from the HTML table header

Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [ ]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
html_tables = soup.find_all('table')

Starting from the third table is our target table contains the actual launch records.


In [ ]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table)

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11"><span class="cite-bracket">[</span>b<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12"><span class="cite-bracket">[</span>c<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [ ]:
column_names = []

# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names
for row in first_launch_table.find_all('th'):
    name = extract_column_from_header(row)
    if name is not None and len(name) > 0:
        column_names.append(name)


Check the extracted column names


In [ ]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


## TASK 3: Create a data frame by parsing the launch HTML tables

We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [ ]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.

To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [ ]:
extracted_row = 0
#Extract each table
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element
        row=rows.find_all('td')
        #if it is number save cells in a dictonary
        if flag:
            extracted_row += 1
            # Flight Number value
            # TODO: Append the flight_number into launch_dict with key `Flight No.`
            launch_dict['Flight No.'].append(flight_number)
            #print(flight_number)
            datatimelist=date_time(row[0])
            print(flight_number)

            # Date value
            # TODO: Append the date into launch_dict with key `Date`
            date = datatimelist[0].strip(',')
            #print(date)
            launch_dict['Date'].append(date)
            print(date)

            # Time value
            # TODO: Append the time into launch_dict with key `Time`
            time = datatimelist[1]
            #print(time)
            launch_dict['Time'].append(time)
            #print(time)
            print(time)

            # Booster version
            # TODO: Append the bv into launch_dict with key `Version Booster`
            #print(bv)
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            launch_dict['Version Booster'].append(bv)
            print(bv)

            # Launch Site
            # TODO: Append the bv into launch_dict with key `Launch Site`
            #print(launch_site)
            launch_site = row[2].a.string
            launch_dict['Launch site'].append(launch_site)
            #print(launch_site)
            print(launch_site)

            # Payload
            # TODO: Append the payload into launch_dict with key `Payload`
            payload = row[3].a.string
            launch_dict['Payload'].append(payload)
            #print(payload)
            print(payload)

            # Payload Mass
            # TODO: Append the payload_mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)
            #print(payload)
            print(payload_mass)

            # Orbit
            # TODO: Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string
            launch_dict['Orbit'].append(orbit)
            #print(orbit)
            print(orbit)

            # Customer
            # TODO: Append the customer into launch_dict with key `Customer`
            #customer = row[6].a.string
            #launch_dict['Customer'].append(customer)
            #customer = row[6].a.string
            #print(customer)
            #print(customer)
             # Comprueba si la etiqueta de anclaje existe y tiene texto antes de acceder a ella
            if row[6].a and row[6].a.string:
                customer = row[6].a.string
            else:
                customer = None  # o algún valor predeterminado si es necesario
            launch_dict['Customer'].append(customer)

            # Launch outcome
            # TODO: Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            launch_dict['Launch outcome'].append(launch_outcome)
            #print(launch_outcome)
            print(launch_outcome)

            # Booster landing
            # TODO: Append the launch_outcome into launch_dict with key `Booster landing`
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)
            #print(booster_landing)
            print(booster_landing)


1
4 June 2010
18:45
F9 v1.07B0003.18
CCAFS
Dragon Spacecraft Qualification Unit
0
LEO
Success

Failure
2
8 December 2010
15:43
F9 v1.07B0004.18
CCAFS
Dragon
0
LEO
Success
Failure
3
22 May 2012
07:44
F9 v1.07B0005.18
CCAFS
Dragon
525 kg
LEO
Success
No attempt

4
8 October 2012
00:35
F9 v1.07B0006.18
CCAFS
SpaceX CRS-1
4,700 kg
LEO
Success

No attempt
5
1 March 2013
15:10
F9 v1.07B0007.18
CCAFS
SpaceX CRS-2
4,877 kg
LEO
Success

No attempt

6
29 September 2013
16:00
F9 v1.17B10038
VAFB
CASSIOPE
500 kg
Polar orbit
Success
Uncontrolled
7
3 December 2013
22:41
F9 v1.1
CCAFS
SES-8
3,170 kg
GTO
Success
No attempt
8
6 January 2014
22:06
F9 v1.1
CCAFS
Thaicom 6
3,325 kg
GTO
Success
No attempt
9
18 April 2014
19:25
F9 v1.1
Cape Canaveral
SpaceX CRS-3
2,296 kg
LEO
Success

Controlled
10
14 July 2014
15:15
F9 v1.1
Cape Canaveral
Orbcomm-OG2
1,316 kg
LEO
Success
Controlled
11
5 August 2014
08:00
F9 v1.1
Cape Canaveral
AsiaSat 8
4,535 kg
GTO
Success
No attempt
12
7 September 2014
05:00
F9 v1.1[
Cape

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [ ]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab.

Following labs will be using a provided dataset to make each lab independent.


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>

In [ ]:
df.head()

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success\n,F9 v1.07B0003.18,Failure,4 June 2010,18:45
1,2,CCAFS,Dragon,0,LEO,NASA,Success,F9 v1.07B0004.18,Failure,8 December 2010,15:43
2,3,CCAFS,Dragon,525 kg,LEO,NASA,Success,F9 v1.07B0005.18,No attempt\n,22 May 2012,07:44
3,4,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success\n,F9 v1.07B0006.18,No attempt,8 October 2012,00:35
4,5,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success\n,F9 v1.07B0007.18,No attempt\n,1 March 2013,15:10


In [ ]:
df.tail()
df.describe(include='all')



,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
count,333,333,333,333,333,327,331,333,331,333,333
unique,121,5,82,84,8,37,3,55,8,121,116
top,1,CCAFS,Starlink,"15,600 kg",LEO,NASA,Success\n,F9 B5,Success,4 June 2010,04:45
freq,3,120,63,50,173,92,232,55,210,3,9


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 333 entries, 0 to 332
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Flight No.       333 non-null    object
 1   Launch site      333 non-null    object
 2   Payload          333 non-null    object
 3   Payload mass     333 non-null    object
 4   Orbit            333 non-null    object
 5   Customer         327 non-null    object
 6   Launch outcome   331 non-null    object
 7   Version Booster  333 non-null    object
 8   Booster landing  331 non-null    object
 9   Date             333 non-null    object
 10  Time             333 non-null    object
dtypes: object(11)
memory usage: 28.7+ KB


In [ ]:
df.to_csv('spacex_web_scraped.csv', index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

Recuerda reemplazar nombre_del_archivo.html con el nombre que desees para tu archivo.

In [ ]:
with open('/content/drive/My Drive/nombre_del_archivo.html', 'w') as f:
    f.write(str(soup))

In [ ]:
import requests
import pandas as pd

In [ ]:
# Especificar la URL de la API de SpaceX:
url = "https://api.spacexdata.com/v4/launches"

In [ ]:
# Realizar la solicitud GET:
response = requests.get(url)

In [ ]:
# Verificar el estado de la solicitud:
if response.status_code == 200:
    print("Solicitud GET exitosa")
else:
    print("Error en la solicitud GET")

Solicitud GET exitosa


In [ ]:
# Convertir la respuesta JSON a un dataframe:
data = response.json()  # Convertir la respuesta a un objeto JSON
df = pd.json_normalize(data)  # Normalizar el JSON y crear el dataframe

##Explicación:

requests.get(url) realiza una solicitud GET a la URL especificada.
response.status_code indica si la solicitud fue exitosa (200) o no.
response.json() convierte la respuesta (que está en formato JSON) a un objeto Python.
pd.json_normalize(data) normaliza el objeto JSON y lo convierte en un dataframe de pandas.

In [ ]:
df.head()


,static_fire_date_utc,static_fire_date_unix,net,window,rocket,success,failures,details,crew,ships,...,links.reddit.media,links.reddit.recovery,links.flickr.small,links.flickr.original,links.presskit,links.webcast,links.youtube_id,links.article,links.wikipedia,fairings
0,2006-03-17T00:00:00.000Z,1.142554e+09,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 33, 'altitude': None, 'reason': 'mer...",Engine failure at 33 seconds and loss of vehicle,[],[],...,None,None,[],[],None,https://www.youtube.com/watch?v=0a_00nJ_Y88,0a_00nJ_Y88,https://www.space.com/2196-spacex-inaugural-fa...,https://en.wikipedia.org/wiki/DemoSat,NaN
1,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 301, 'altitude': 289, 'reason': 'har...",Successful first stage burn and transition to ...,[],[],...,None,None,[],[],None,https://www.youtube.com/watch?v=Lk4zQ2wP-Nc,Lk4zQ2wP-Nc,https://www.space.com/3590-spacex-falcon-1-roc...,https://en.wikipedia.org/wiki/DemoSat,NaN
2,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 140, 'altitude': 35, 'reason': 'resi...",Residual stage 1 thrust led to collision betwe...,[],[],...,None,None,[],[],None,https://www.youtube.com/watch?v=v0w9p3U8860,v0w9p3U8860,http://www.spacex.com/news/2013/02/11/falcon-1...,https://en.wikipedia.org/wiki/Trailblazer_(sat...,NaN
3,2008-09-20T00:00:00.000Z,1.221869e+09,False,0.0,5e9d0d95eda69955f709d1eb,True,[],Ratsat was carried to orbit on the first succe...,[],[],...,None,None,[],[],None,https://www.youtube.com/watch?v=dLQ2tZEH6G0,dLQ2tZEH6G0,https://en.wikipedia.org/wiki/Ratsat,https://en.wikipedia.org/wiki/Ratsat,NaN
4,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,True,[],None,[],[],...,None,None,[],[],http://www.spacex.com/press/2012/12/19/spacexs...,https://www.youtube.com/watch?v=yTaIDooc8Og,yTaIDooc8Og,http://www.spacex.com/news/2013/02/12/falcon-1...,https://en.wikipedia.org/wiki/RazakSAT,NaN


In [ ]:
df.columns

Index(['static_fire_date_utc', 'static_fire_date_unix', 'net', 'window',
       'rocket', 'success', 'failures', 'details', 'crew', 'ships', 'capsules',
       'payloads', 'launchpad', 'flight_number', 'name', 'date_utc',
       'date_unix', 'date_local', 'date_precision', 'upcoming', 'cores',
       'auto_update', 'tbd', 'launch_library_id', 'id', 'fairings.reused',
       'fairings.recovery_attempt', 'fairings.recovered', 'fairings.ships',
       'links.patch.small', 'links.patch.large', 'links.reddit.campaign',
       'links.reddit.launch', 'links.reddit.media', 'links.reddit.recovery',
       'links.flickr.small', 'links.flickr.original', 'links.presskit',
       'links.webcast', 'links.youtube_id', 'links.article', 'links.wikipedia',
       'fairings'],
      dtype='object')

After you performed a GET request on the Space X API and convert the response to a dataframe using pd.json_normalize. What year is located in the first row in the column static_fire_date_utc?

Primero necesitamos acceder a la columna static_fire_date_utc en la primera fila del dataframe. Luego, extraemos el año de la fecha que se encuentra en esa celda.

Convierte la cadena de fecha a un objeto de fecha usando la librería datetime:

La clave rocket.name no está directamente en el dataframe que se crea con pd.json_normalize(data).

La información del cohete está anidada dentro de la columna rocket, que contiene un ID. Para acceder al nombre del cohete, primero necesitas usar ese ID para obtener la información del cohete de la API de SpaceX, y luego buscar el nombre en los datos del cohete.

Explicación de los cambios:

Función get_rocket_name: Esta función toma el ID del cohete como entrada, realiza una solicitud GET a la API de cohetes de SpaceX para obtener información sobre ese cohete y devuelve el nombre del cohete.
df['rocket_name'] = df['rocket'].apply(get_rocket_name): Se aplica la función get_rocket_name a la columna rocket del dataframe para crear una nueva columna llamada rocket_name que contiene el nombre de cada cohete.
Filtro y conteo: El resto del código funciona igual que antes, filtrando por 'Falcon 9' en la nueva columna rocket_name y contando las filas.

In [ ]:
import requests
import pandas as pd

# Función para obtener el nombre del cohete por su ID
def get_rocket_name(rocket_id):
    rocket_url = f"https://api.spacexdata.com/v4/rockets/{rocket_id}"
    rocket_response = requests.get(rocket_url)
    if rocket_response.status_code == 200:
        rocket_data = rocket_response.json()
        return rocket_data['name']
    else:
        return None

# Realizar la solicitud GET a la API de lanzamientos
url = "https://api.spacexdata.com/v4/launches"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    df = pd.json_normalize(data)

    # Obtener el nombre del cohete para cada lanzamiento
    df['rocket_name'] = df['rocket'].apply(get_rocket_name)

    # Filtrar para obtener solo lanzamientos de Falcon 9
    falcon9_launches = df.loc[df['rocket_name'] == 'Falcon 9']

    # Contar el número de lanzamientos de Falcon 9
    num_falcon9_launches = falcon9_launches.shape[0]
    print("Número de lanzamientos de Falcon 9:", num_falcon9_launches)

else:
    print("Error en la solicitud GET")

Número de lanzamientos de Falcon 9: 195


At the end of the API data collection process, how many missing values are there for the column landingPad?




necesitamos verificar la estructura actual de los datos JSON y ajustar la forma en que desanidamos la columna 'cores' para capturar la información de 'landing_attempt'.

Explicación de los cambios:

Desanidación precisa: En lugar de desanidar 'cores' directamente, se usa pd.json_normalize con record_path=['cores'] y meta=['flight_number'] para acceder a 'landing_attempt' dentro de la estructura de 'cores' y mantener la referencia al 'flight_number' para la unión posterior.
Extracción y Renombramiento: Se crea un nuevo DataFrame landing_attempt_df con las columnas 'flight_number' y 'landing_attempt', y se renombra 'landing_attempt' a 'cores.0.landing_attempt'.
Unión por 'flight_number': Se usa pd.merge para unir landing_attempt_df al DataFrame original df usando 'flight_number' como clave, asegurando que la información de 'landing_attempt' se asocie correctamente a cada lanzamiento.

In [ ]:
import requests
import pandas as pd

# Realiza la solicitud GET a la API de lanzamientos
url = "https://api.spacexdata.com/v4/launches"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()

    # Crea el dataframe inicial
    df = pd.json_normalize(data)

    # Desanida la columna 'cores', accediendo a 'landing_attempt'
    cores_df = pd.json_normalize(data, record_path=['cores'], meta=['flight_number'], errors='ignore')
    # Extraer solo la columna 'landing_attempt'
    landing_attempt_df = cores_df[['flight_number', 'landing_attempt']]
    # Renombrar la columna 'landing_attempt'
    landing_attempt_df = landing_attempt_df.rename(columns={'landing_attempt': 'cores.0.landing_attempt'})

    # Unir la columna 'cores.0.landing_attempt' al DataFrame original usando 'flight_number'
    df = pd.merge(df, landing_attempt_df, on='flight_number', how='left')

    # Calcula la cantidad de valores faltantes en 'cores.0.landing_attempt'
    missing_values = df['cores.0.landing_attempt'].isnull().sum()
    print("Cantidad de valores faltantes en landingPad:", missing_values)

else:
    print("Error en la solicitud GET")

Cantidad de valores faltantes en landingPad: 42


Repetimos ejercicios



Pregunta 1: ¿Cuántos lanzamientos de Falcon 9 hay después de eliminar los lanzamientos de Falcon 1?

In [ ]:
import requests
import pandas as pd

# Función para obtener el nombre del cohete por su ID
def get_rocket_name(rocket_id):
    rocket_url = f"https://api.spacexdata.com/v4/rockets/{rocket_id}"
    rocket_response = requests.get(rocket_url)
    if rocket_response.status_code == 200:
        rocket_data = rocket_response.json()
        return rocket_data['name']
    else:
        return None

# Realizar la solicitud GET a la API de lanzamientos
url = "https://api.spacexdata.com/v4/launches"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    df = pd.json_normalize(data)

    # Obtener el nombre del cohete para cada lanzamiento
    df['rocket_name'] = df['rocket'].apply(get_rocket_name)

    # Filtrar para obtener solo lanzamientos de Falcon 9
    falcon9_launches = df[df['rocket_name'] == 'Falcon 9']

    # Contar el número de lanzamientos de Falcon 9
    num_falcon9_launches = falcon9_launches.shape[0]
    print("Número de lanzamientos de Falcon 9:", num_falcon9_launches)

else:
    print("Error en la solicitud GET")

Número de lanzamientos de Falcon 9: 195


Explicación paso a paso:

Importar librerías: Se importan requests para hacer la solicitud a la API y pandas para manejar los datos.
Función get_rocket_name: Esta función toma el ID del cohete y hace una solicitud a la API de SpaceX para obtener su nombre.
Solicitud a la API: Se realiza la solicitud GET a la API de lanzamientos de SpaceX.
Creación del DataFrame: Si la solicitud es exitosa, se crea un DataFrame df con los datos de los lanzamientos.
Obtener el nombre del cohete: Se aplica la función get_rocket_name a la columna 'rocket' del DataFrame para obtener el nombre de cada cohete y se guarda en una nueva columna 'rocket_name'.
Filtrar por Falcon 9: Se filtra el DataFrame para quedarse solo con los lanzamientos de Falcon 9.
Contar lanzamientos: Se calcula y se imprime el número de lanzamientos de Falcon 9.

Pregunta 2: ¿Cuántos valores faltantes hay en la columna 'landingPad' al final del proceso de recolección de datos de la API?
Para esta pregunta, el problema principal era que la columna landingPad no se encuentra directamente en la respuesta de la API. Necesitamos acceder a ella a través de la información de los 'cores' (núcleos del cohete).

In [ ]:
import requests
import pandas as pd

# Realiza la solicitud GET a la API de lanzamientos
url = "https://api.spacexdata.com/v4/launches"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()

    # Crea el dataframe inicial
    df = pd.json_normalize(data)

    # Desanida la columna 'cores' para acceder a 'landing_pad'
    cores_df = pd.json_normalize(data, record_path=['cores'], meta=['flight_number'])

    # Extraer 'landing_pad' usando una función lambda que busca en los diccionarios anidados
    cores_df['landing_pad'] = cores_df.apply(lambda row: row.get('landpad') or (row[0].get('landpad') if isinstance(row[0], dict) else None) if 'landpad' in row or (isinstance(row[0], dict) and 'landpad' in row[0]) else None, axis=1)

    # Unir la columna 'landing_pad' al DataFrame original usando 'flight_number'
    df = pd.merge(df, cores_df[['flight_number', 'landing_pad']], on='flight_number', how='left')

    # Calcula la cantidad de valores faltantes en 'landing_pad'
    missing_values = df['landing_pad'].isnull().sum()
    print("Cantidad de valores faltantes en landingPad:", missing_values)

else:
    print("Error en la solicitud GET")

Cantidad de valores faltantes en landingPad: 78


<ipython-input-47-e6989f91efc4>:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cores_df['landing_pad'] = cores_df.apply(lambda row: row.get('landpad') or (row[0].get('landpad') if isinstance(row[0], dict) else None) if 'landpad' in row or (isinstance(row[0], dict) and 'landpad' in row[0]) else None, axis=1)


In [ ]:
import requests
import pandas as pd

# Realizar la solicitud GET a la API de lanzamientos
url = "https://api.spacexdata.com/v4/launches"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    df = pd.json_normalize(data)

    # Filtrar los lanzamientos desde CCAFS SLC 40
    ccafs_slc_40_launches = df[df['launchpad'] == '5e9e4502f5090995de566f86']

    # Contar el número de lanzamientos
    num_ccafs_slc_40_launches = ccafs_slc_40_launches.shape[0]

    print("Número de lanzamientos desde CCAFS SLC 40:", num_ccafs_slc_40_launches)
else:
    print("Error en la solicitud GET")

Número de lanzamientos desde CCAFS SLC 40: 5


In [ ]:
import requests
import pandas as pd

# Realizar la solicitud GET a la API de lanzamientos
url = "https://api.spacexdata.com/v4/launches"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    df = pd.json_normalize(data)

    # Filtrar los lanzamientos desde CCAFS SLC 40
    ccafs_slc_40_launches = df[df['launchpad'] == '5e9e4502f5090995de566f86']

    # Contar el número de lanzamientos exitosos
    successful_launches = ccafs_slc_40_launches[ccafs_slc_40_launches['success'] == True].shape[0]

    # Calcular el porcentaje de tasa de éxito
    success_rate = (successful_launches / ccafs_slc_40_launches.shape[0]) * 100

    print("Tasa de éxito de los lanzamientos desde CCAFS SLC 40: {:.2f}%".format(success_rate))
else:
    print("Error en la solicitud GET")

Tasa de éxito de los lanzamientos desde CCAFS SLC 40: 40.00%


In [ ]:
import requests
import pandas as pd

url = "https://api.spacexdata.com/v4/launches"
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
else:
    print("Error en la solicitud GET")

In [ ]:
df = pd.json_normalize(data)

In [ ]:
cores_df = pd.json_normalize(data, record_path=['cores'], meta=['flight_number'])

In [ ]:
failed_landings = cores_df[
    (cores_df['landing_attempt'] == True) & (cores_df['landing_success'] == False)
]

In [ ]:
num_failed_landings = len(failed_landings)
print("Número de aterrizajes fallidos:", num_failed_landings)

Número de aterrizajes fallidos: 13
